In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u 

from snewpy.neutrino import Flavor, MassHierarchy, MixingParameters
from snewpy.models.ccsn import Nakazato_2013
from snewpy.flavor_transformation import AdiabaticMSW, NonAdiabaticMSWH, \
                                         TwoFlavorDecoherence, ThreeFlavorDecoherence, \
                                         NeutrinoDecay, AdiabaticMSWes, NonAdiabaticMSWes

mpl.rc('font', size=18)
%matplotlib inline

In [2]:
model = Nakazato_2013(progenitor_mass=20*u.solMass, revival_time=100*u.ms, metallicity=0.004, eos='shen')
model

**Nakazato_2013 Model**: nakazato-shen-z0.004-t_rev100ms-s20.0.fits

|Parameter|Value|
|:--------|:----:|
|Progenitor mass | $20$ $\mathrm{M_{\odot}}$|
|Revival time | $100$ $\mathrm{ms}$|
|Metallicity | 0.004 |
|EOS | shen |

In [3]:
D=10*u.kpc
c = 299792458 * u.m / u.s
tof_light=D.to(u.m)/c

T=(100* u.ms)
    
E = 30* u.MeV
d = (10*u.kpc).to('cm').value # distance to SN
    
#M=np.linspace(0.1,1,5)*u.eV
M=1* u.eV
   
V= D.to(u.m).value/(c.value*(1-(M.value/ E.to(u.eV).value)**2)**0.5)-D.to(u.m).value/c.value
print(V)

0.0006103515625


In [4]:
def plot_spectra12(model, xform_nmo, xform_imo, t):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    t : astropy.Quantity
        Time to compute the spectrum.
    """

    energies = np.linspace(1,50,5000) * u.MeV   
    D=10* u.kpc
    d = (10*u.kpc).to('cm').value # distance to SN
    mass=1 *u.eV    
    #get the spectra
    ospec_nmo = model.get_transformed_spectra(t, energies, xform_nmo)
    ospec_imo = model.get_transformed_spectra(t, energies, xform_imo)
    ospec_nmo_tof,tof1 = model.get_delayed_flux(D, t, energies, mass, xform_nmo)
    ospec_imo_tof,tof2= model.get_delayed_flux(D, t, energies, mass, xform_imo)
    

    for flavor in Flavor:
        for j, E in enumerate(energies):
            
            ospec_nmo[flavor][j] /= (4.*np.pi*d**2)
            ospec_imo[flavor][j] /= (4.*np.pi*d**2)
            ospec_nmo_tof[flavor][j] /= (4.*np.pi*d**2)
            ospec_imo_tof[flavor][j] /= (4.*np.pi*d**2)
    
    energy_bins = [(1, 10), (10, 30), (30, 50)]
    energy_values = energies.value
    fig, axes = plt.subplots(2,3, figsize=(24,21), tight_layout=True)
    titles = ['nmo_tof','imo_tof']
    smax = [0.,0.,0.]
    for i, spec in enumerate([ospec_nmo_tof, ospec_imo_tof]):
        for j, E in enumerate(energies):
            ax = axes[i,j]
            
            mask = (energy_values >= energy_min) & (energy_values < energy_max)
            
            for flavor in Flavor:
                if i == 0:
                    smax[j] = np.maximum(smax[j], 1.1 * np.max(spec[flavor][mask]))
                # 在子图上绘制光度曲线
                ax.plot(energy_values[mask],  # 使用筛选后的能量数据
                    spec[flavor][mask],  # 光度数据
                    label=flavor.to_tex(),  # 图例标签
                    lw=3,  # 线宽
                    color='C0' if flavor.is_electron else 'C1',  # 颜色
                    ls='-' if flavor.is_neutrino else ':')  # 线型
             # 设置子图的x轴和y轴范围
            ax.set(xlim=(energy_min, energy_max),
                   ylim=(0, smax[j]))
            
            # 如果是第一个能量范围，设置y轴标签和图例
            if j == 0:
                ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
                ax.legend(loc='upper right', ncol=1, fontsize=18)
            
            # 如果是第二列，设置子图标题
            if j == 1:
                ax.set(title=titles[i])
            
            # 如果不是最后一行，隐藏x轴标签
            if i < 2:
                ax.set(xticklabels=[])
            else:
                # 否则，设置x轴标签
                ax.set(xlabel='Energy [MeV]')
            
            # 设置网格线
            ax.grid(ls=':')
                
    return fig,tof1,ospec_nmo_tof

In [ ]:
fig,tof1,ospec_nmo_tof= plot_spectra12(model, AdiabaticMSW(), AdiabaticMSW(mh=MassHierarchy.INVERTED), 1000*u.ms)
print(tof1)
print(ospec_nmo_tof[Flavor.NU_E])
print(np.max(ospec_nmo_tof[Flavor.NU_E]))
print(np.argmax(ospec_nmo_tof[Flavor.NU_E]))